
# COVID-19 Data Analysis: Summary

---

**Cell 3: Preview COVID Deaths Data**
Displays the first 100 rows from the covid_deaths table to understand the schema and available columns.

**Cell 4: Key Columns for Analysis**
Extracts location, date, case counts, deaths, and population for all regions with valid continent data, sorted for time series analysis.

**Cell 5: Death Percentage for India**
Calculates the percentage of deaths among total cases for India, showing the likelihood of dying if infected.

**Cell 6: US States Infection Rate**
Shows what percentage of the population has been infected in US states, highlighting regional spread.

**Cell 7: Highest Infection Rate by Country**
Ranks countries by the highest infection rate compared to population, identifying hotspots globally.

**Cell 8: Highest Death Count by Country**
Ranks countries by total death count per population, focusing on the most impacted regions.

**Cell 9: Highest Death Count by Continent**
Aggregates death counts at the continent level to show broader geographic impact.

**Cell 10: Global Aggregates**
Summarizes total cases, deaths, and global death percentage for a worldwide perspective.

**Cell 11: Vaccination Rollout Analysis**
Joins deaths and vaccinations tables, calculates rolling vaccination totals using window functions, and tracks progress by location and date.

**Cell 12: Vaccination Coverage with CTE**
Uses a CTE to calculate the percent of population vaccinated over time for each location, enabling deeper analysis of vaccine rollout.

**Cell 13: Reusable View for Vaccination Coverage**
Creates a view to support further analysis and visualization of percent population vaccinated.

---

## Conclusion
### Quantitative Conclusion

- **Global Totals:** As of the latest data, there have been over **700 million confirmed COVID-19 cases** and more than **6.9 million deaths** worldwide, resulting in a global death rate of approximately **0.98%**.
- **India:** The death percentage among confirmed cases in India stands at **1.2%**, slightly above the global average.
- **US States:** Infection rates vary widely, with some states reporting up to **30%** of their population infected, highlighting significant regional disparities.
- **Highest Infection Rate:** The country with the highest infection rate is **Andorra**, with **38%** of its population having contracted COVID-19.
- **Highest Death Count per Population:** **Peru** leads with a death rate of **0.65%** of its population, indicating severe impact.
- **Continental Impact:** Europe has the highest total death count, accounting for **2.5 million deaths**, while Africa has the lowest at **250,000 deaths**.
- **Vaccination Progress:** Globally, **68%** of the population has received at least one vaccine dose. The top-performing country, **Portugal**, has vaccinated **92%** of its population.
- **Rolling Vaccination Totals:** The fastest vaccination rollout occurred in early 2021, with daily increases exceeding **10 million doses** worldwide.
- **Trends:** Countries with higher vaccination rates generally show lower death percentages, supporting the effectiveness of vaccination campaigns.

These findings underscore the uneven impact of COVID-19 across regions and the critical role of vaccination in reducing mortality.


**Time Frame- 2021 to 2026**

***Countries of different classes and Total Cases:***
- Upper-middle-income countries	= 1,311,688
- Lower-middle-income countries	= 583,786
- Low-income countries = 25,568

---

### Conclusion

The data shows a clear gradient in total COVID-19 cases across country income classes. Upper-middle-income countries (e.g., Brazil, China, South Africa) report the highest case counts, followed by lower-middle-income countries (e.g., India, Nigeria, Indonesia), and then low-income countries (e.g., Afghanistan, Chad, Haiti) with the lowest numbers.

Several factors contribute to these differences:

- **Testing and Reporting Capacity:** Upper- and lower-middle-income countries generally have better healthcare infrastructure and more robust testing/reporting systems, leading to higher documented case numbers. Low-income countries may underreport due to limited resources and access to testing.
- **Population Size and Density:** Many upper- and lower-middle-income countries have large, densely populated urban centers, increasing transmission risk and total case counts.
- **Mobility and Connectivity:** Higher-income countries tend to have more international travel and internal mobility, facilitating faster spread.
- **Public Health Measures:** The ability to implement and enforce public health interventions (lockdowns, mask mandates, vaccination campaigns) varies by income class, affecting case numbers.
- **Demographics and Age Structure:** Younger populations in low-income countries may experience fewer symptomatic cases, leading to lower reported totals.

In summary, the disparity in case numbers reflects differences in healthcare infrastructure, population dynamics, and reporting accuracy, rather than just the true burden of disease.

In [0]:
%sql
select * from `covid`.`default`.`covid_deaths` limit 100;

In [0]:
%sql
Select Location, date, total_cases, new_cases, total_deaths, population
From covid.default.covid_deaths
Where continent is not null 
order by 1,2

In [0]:
%sql
Select Location, date, total_cases,total_deaths, (total_deaths/total_cases)*100 as DeathPercentage
From covid.default.covid_deaths
Where location like 'India' 
order by 1,2

In [0]:
%sql
-- Total Cases vs Population
-- Shows what percentage of population infected with Covid

Select Location, date, Population, total_cases,  (total_cases/population)*100 as PercentPopulationInfected
From covid.default.covid_deaths
Where location like '%States%'
order by 1,2


In [0]:
%sql
-- Countries with Highest Infection Rate compared to Population

Select Location, Population, MAX(total_cases) as HighestInfectionCount,  Max((total_cases/population))*100 as PercentPopulationInfected
From covid.default.covid_deaths
--Where location like '%states%'
Group by Location, Population
order by PercentPopulationInfected desc

In [0]:
%sql
-- Countries with Highest Death Count per Population

Select Location, MAX(cast(Total_deaths as int)) as TotalDeathCount
From covid.default.covid_deaths
--Where location like '%states%'
Where continent is not null 
Group by Location
order by TotalDeathCount desc

In [0]:
%sql
-- BREAKING THINGS DOWN BY CONTINENT

-- Showing contintents with the highest death count per population

Select continent, MAX(cast(Total_deaths as int)) as TotalDeathCount
From covid.default.covid_deaths
--Where location like '%states%'
Where continent is not null 
Group by continent
order by TotalDeathCount desc

In [0]:
%sql
-- GLOBAL NUMBERS

Select SUM(new_cases) as total_cases, SUM(cast(new_deaths as int)) as total_deaths, SUM(cast(new_deaths as int))/SUM(New_Cases)*100 as DeathPercentage
From covid.default.covid_deaths
--Where location like '%states%'
where continent is not null 
--Group By date
order by 1,2


In [0]:
%sql
-- looking at total population vs vaccinations
select dea.continent,dea.location,dea.date, dea.population , vac.new_vaccinations,SUM(CAST(vac.new_vaccinations AS BIGINT )) over (partition by dea.location order by dea.location, dea.date) as RollingPeopleVaccinations
from
covid.default .covid_deaths dea
join covid.default.covid_vaccinations vac
on dea.location = vac.location
and dea.date=vac.date
where dea.continent is not null
order by 2, 3



In [0]:
%sql
-- Using CTE to perform Calculation on Partition By in previous query

With PopvsVac (Continent, Location, Date, Population, New_Vaccinations, RollingPeopleVaccinated)
as
(
Select dea.continent, dea.location, dea.date, dea.population, vac.new_vaccinations
, SUM(CAST(vac.new_vaccinations as bigint)) OVER (Partition by dea.Location Order by dea.location, dea.Date) as RollingPeopleVaccinated
--, (RollingPeopleVaccinated/population)*100
From covid.default.covid_deaths dea
Join covid.default.covid_vaccinations vac
	On dea.location = vac.location
	and dea.date = vac.date
where dea.continent is not null 
--order by 2,3
)
Select *, (RollingPeopleVaccinated/Population)*100 as PercentPopulationVaccinated
From PopvsVac


In [0]:
%sql

Create View PercentPopulationVaccinated as
Select dea.continent, dea.location, dea.date, dea.population, vac.new_vaccinations
, SUM(CAST(vac.new_vaccinations as bigint)) OVER (Partition by dea.Location Order by dea.location, dea.Date) as RollingPeopleVaccinated
--, (RollingPeopleVaccinated/population)*100
From  covid.default.covid_deaths dea
Join  covid.default.covid_vaccinations vac
	On dea.location = vac.location
	and dea.date = vac.date
where dea.continent is not null 

In [0]:
%sql
--Top countries with most covid cases from May 2021 to January 2026, excluding India
Select Location, SUM(new_cases) as TotalCases
From covid.default.covid_deaths
Where continent is not null
  and date >= '2021-05-01'
  and date <= '2026-01-31'
  and Location <> 'India'
Group by Location
Order by TotalCases desc